In [272]:
import pandas as pd
import re

Carga de datos CSV 

In [273]:
#Carga de datos CSV 

#Carga de Producto
df_Producto = pd.read_csv("L_STMaestroPrd.csv")
# Eliminar filas donde todas sus columnas están vacías
df_MasterProductos = df_Producto.dropna(how='all')

# Eliminar columnas donde todos sus valores están vacíos
df_MasterProductos = df_MasterProductos.dropna(axis=1, how='all')
df_MasterProductos["descripcion_limpia_producto"] = df_MasterProductos["descrip1"].fillna('') + " " + df_MasterProductos["descrip2"].fillna('')

#Carga Proveedor
df_Proveedor = pd.read_csv('Phoenix.csv')
# Eliminar filas donde todas sus columnas están vacías
df_Proveedor = df_Proveedor.dropna(how='all')

# Eliminar columnas donde todos sus valores están vacíos
df_Proveedor = df_Proveedor.dropna(axis=1, how='all')

df_Proveedor['descripcion_limpia_proveedor'] = df_Proveedor['descripcion']

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_14988\2265950528.py:4: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Producto = pd.read_csv("L_STMaestroPrd.csv")


Estandarizacion de COLUMNAS

In [274]:
#Funcion que normaliza COLUMNAS
def normalizar_columnas_y_unir (df, col1, col2, nueva_columna="descripcion_limpia_proveedor"):
    """
    Normaliza los nombres de las columnas a minúsculas y une dos columnas específicas si existen.
    
    Args:
        df (pd.DataFrame): DataFrame a procesar.
        col1 (str): Nombre de la primera columna a unir.
        col2 (str): Nombre de la segunda columna a unir.
        nueva_columna (str): Nombre de la nueva columna que contendrá la unión.
    
    Returns:
        pd.DataFrame: DataFrame con las columnas normalizadas y la nueva columna creada.
    """
    # Normalizar los nombres de las columnas a minúsculas
    df.columns = df.columns.str.lower()

    # Usar `str.cat()` para concatenar las columnas con un espacio.
    df[nueva_columna] = df[col1].astype(str).str.cat(df[col2].astype(str), sep=' ', na_rep='').str.strip()
    
    return df


#Normalizamos las Columnas del Proveedor
df_Proveedor = normalizar_columnas_y_unir(df_Proveedor, 'descripcion', 'presentacion')

In [275]:
def procesar_descripcion(data, nombre_columna):
    """
    Procesa una columna de un DataFrame aplicando normalización y manejo de combinaciones de texto.
    
    Parámetros:
    - data: DataFrame que contiene la columna a procesar.
    - nombre_columna: Nombre de la columna que se va a procesar.
    
    Retorna:
    - DataFrame con la columna procesada y actualizada.
    """
    # Verificar si la columna existe en el DataFrame
    if nombre_columna not in data.columns:
        raise KeyError(f"La columna '{nombre_columna}' no existe en el DataFrame.")
    
    # Función para normalizar el texto
    def normalizar(texto):
        # Asegurar que el texto sea un string
        texto = str(texto).strip().lower()
        
        # 1. Eliminar espacios redundantes
        texto = re.sub(r'\s+', ' ', texto)

        # 2. Remover caracteres especiales excepto letras, números, espacios y '%'
        texto = re.sub(r'[^a-zA-Z0-9\s%]', '', texto)

        # 3. Separar números de letras y letras de números
        texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
        texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)

        # 4. Manejo de 'x':
        texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)  # separar x entre números
        texto = re.sub(r'\b([a-zA-Z]+)x\b', r'\1 x', texto)  # separar palabras terminadas en x

        # 5. Reemplazar separadores como guiones o barras con espacios
        texto = re.sub(r'[-/]', ' ', texto)

        # 6. Manejo de unidades (por ejemplo, separar g, ml, etc. de los números)
        texto = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', texto)

        # 7. Eliminar espacios redundantes nuevamente
        texto = texto.strip()

        # 8. Combinaciones
        combinaciones = {
            'anemido x': 'anemidox',
            'jgaprell' : 'jga prell',
            'fle x': 'flex',
            'argeflo x':'argeflox',
            'aropa x': 'aropax',
            'arolte x' : 'aroltex',
            'atopi x':  'atopix',
            'atorma x': 'atormax',
            'comprec': 'comp recubierto',
            'soloft': 'solucion oftalmica',
            'g': 'gr',
            'cr': 'crema',
            'comp' : 'comprimidos',
            'caps': 'capsulas',
            "jer.pre" : "jeringa prell",
            "mg" : "miligramos",
            "ml" : "mililitros",
            "mm" : "milimetros",
            'zyvo x': 'zyvox',
            'loc' : 'locion',
            "gr" : "gramos",
            "jga" : 'jeringa',
            "cm" : 'centimetros',
            'u' : "unidad",
            'shamp' : 'shampoo',
            "unid" : " unidad",
            'cep' : 'cepillo dental',
            "emuls prot" : 'emulsion protectora',
            'emulshidrat':'emulsion hidratante',
            'emulshumect' : "emulsion humectante",
            'iny': 'inyeccion',
            'amp' : 'ampolla',
            'inylioffa': 'inyeccion lioffa',
            'ds' : 'dosis',
            'inyfa': 'inyeccion fa',
            'jbe': 'jarabe',
            'blist': 'blister',
            'sob' : 'sobres',
            'fa': 'frasco ampolla',
            'jab' : 'jabon',
            'cps' :'capsulas',
            'ivi x' : 'ivix',
            'inya': 'inyectable',
            'inyjgaprell' : 'iny jeringa prellenada',
            'mcg': 'microgramo',
            'gts': 'gotas',
            'mgml' : 'miligramos por mililitro',
            'env': 'envase',
            'gtsoft' : 'gotas oftálmicas',
            'compdisp': 'comprimidos disp',
            'comprecran' : 'comprimidos recubiertos ranurados',
            'sol': 'solucion',
            'jgapre': 'jeringa prellenada',
            'fcocps' : 'frasco capsulas',
            'fcogotero' : 'frasco gotero',
            'fco': 'frasco', 
            'solspray' : 'solucion spray',
            'solpuas' : 'sol puas',
            'cpsblandas' : 'cps blandas',
            'tab' : 'tableta',
            'sach' : 'sachet',
            'mgvial': 'miligramos vial',
            'cpsbl' :'capsula blanda'
        }
        for key, val in combinaciones.items():
            texto = re.sub(rf'\b{key}\b', val, texto)

        return texto

    # Aplicar la función 'normalizar' a cada elemento de la columna
    data[nombre_columna] = data[nombre_columna].apply(normalizar)
    
    return data  # Retorna el DataFrame con la columna procesada


In [276]:
#Procesamiento de Columna Descripcion

# Procesamiento de df_MasterProductos: descripcion_limpia_producto
df_MasterProductos = procesar_descripcion(df_MasterProductos, "descripcion_limpia_producto")

# Procesamiento de df_Proveedor: descripcion_limpia_proveedor
df_Proveedor = procesar_descripcion(df_Proveedor, 'descripcion_limpia_proveedor')



In [277]:
df_MasterProductos['descripcion_limpia_producto']

0                                                         
1                                                         
2        repetido supportan 500 mililitros caja x 15 sa...
3        bolsas papel grado medico 175 x 65 x 420 milim...
4        bolsas papel grado medico 260 x 60 x 520 milim...
                               ...                        
56769            zyvali x 250 miligramos comprimidos x 120
56770        zyvali x plus 500 miligramos comprimidos x 60
56771    zyvox iv 600 miligramos bolsas x 10 x 300 mili...
56772               zyvox oral 600 miligramos tableta x 10
56773                              zz top comprimidos x 30
Name: descripcion_limpia_producto, Length: 56770, dtype: object

In [319]:
# Función para filtrar los productos según el laboratorio ingresado (sin importar mayúsculas/minúsculas)
def filtrar_por_laboratorio(df, laboratorio_input, columna_laboratorio='atrib0'):
    # Convertir tanto la columna de laboratorios como el input a minúsculas para hacer la comparación insensible a mayúsculas
    df_filtrado = df[df[columna_laboratorio].str.lower() == laboratorio_input.lower()]
    if not df_filtrado.empty:
        print(f"Se encontraron {len(df_filtrado)} productos coincidentes del laboratorio '{laboratorio_input}'.")
        return df_filtrado
    else:
        print(f"No se encontraron productos del laboratorio '{laboratorio_input}'.")
        return None

# Pedir al usuario que ingrese el nombre del laboratorio
laboratorio_input = input("Ingresa el nombre del laboratorio: ")  # El usuario ingresa el nombre del laboratorio

# Filtrar los productos correspondientes a ese laboratorio
df_MasterProductos_Filtro = filtrar_por_laboratorio(df_MasterProductos, laboratorio_input)

# Ver el resultado del dataframe filtrado
#df_MasterProductos_Filtro.columns


Se encontraron 402 productos coincidentes del laboratorio 'Phoenix'.


In [279]:
df_MasterProductos_Filtro

,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descripcion_limpia_producto
604,26458.0,33228,33228,ACOTRAL 10 mg comp.x 10,NaN,ACOTRAL 10 mg comp.x,PT,7791848286017,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,acotral 10 miligramos comprimidos x 10
605,26459.0,33229,33229,ACOTRAL 10 mg comp.x 20,NaN,ACOTRAL 10 mg comp.x,PT,7791848286024,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,acotral 10 miligramos comprimidos x 20
606,26460.0,33751,33751,ACOTRAL 10 mg comp.x 30,NaN,ACOTRAL 10 mg comp.x,PT,7791848286031,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,acotral 10 miligramos comprimidos x 30
1266,26461.0,33082,33082,AGIOFIBRAS gran.x 250 g,NaN,AGIOFIBRAS gran.x 25,PT,7791848259066,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,agiofibras gran x 250 gramos
1267,26462.0,46479,46479,AGIOFIBRAS sob.x 12,NaN,AGIOFIBRAS sob.x 12,PT,7791848259097,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,agiofibras sobres x 12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,91314.0,23342,23342,XANOL 12 mcg caps.x 30,NaN,XANOL 12 mcg caps.x,PT,7791848262028.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,xanol 12 microgramo capsulas x 30
56038,91315.0,25874,25874,XANOL 12 mcg caps.x 60,NaN,XANOL 12 mcg caps.x,PT,7791848262042.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,xanol 12 microgramo capsulas x 60
56039,91316.0,23341,23341,XANOL 6 mcg caps.x 30,NaN,XANOL 6 mcg caps.x 3,PT,7791848262011.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,xanol 6 microgramo capsulas x 30
56040,91317.0,25873,25873,XANOL 6 mcg caps.x 60,NaN,XANOL 6 mcg caps.x 6,PT,7791848262035.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,xanol 6 microgramo capsulas x 60


Función de comparación con similitud de coseno


In [280]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Vectorizar todas las descripciones (tanto de la master como del proveedor)
descripciones_completas = df_MasterProductos_Filtro['descripcion_limpia_producto'].tolist() + df_Proveedor['descripcion_limpia_proveedor'].tolist()
vectorizer = TfidfVectorizer().fit_transform(descripciones_completas)

# Calcular similitud de coseno entre las descripciones de la master y del proveedor
cos_sim_matrix = cosine_similarity(vectorizer[:len(df_MasterProductos_Filtro)], vectorizer[len(df_MasterProductos_Filtro):])

# Para cada producto del proveedor, obtener la similitud más alta con los productos de la master
matches = []

# Asegurarse de que el índice de las filas del proveedor no exceda el número de productos en df_Proveedor
for i, prov_row in df_Proveedor.iterrows():
    prov_info = prov_row.to_dict()  # Información completa del producto del proveedor
    
    # Obtener las similitudes entre el producto del proveedor y todos los productos de la master
    if i < cos_sim_matrix.shape[1]:  # Asegurarse de que i no exceda el número de columnas
        similitudes = cos_sim_matrix[:, i]  # Similitudes entre el proveedor i y todos los productos de la master
        
        # Encontrar la mejor coincidencia (la mayor similitud)
        max_similitud = np.max(similitudes)  # La mayor similitud encontrada
        best_match_index = np.argmax(similitudes)  # Índice del producto de la master con la mayor similitud
        
        # Recuperar la fila de la master correspondiente a la mejor coincidencia
        best_match_info = df_MasterProductos_Filtro.iloc[best_match_index].to_dict()
        
        # Unir la información del proveedor y la mejor coincidencia
        best_match_info.update(prov_info)
        best_match_info['similaridad'] = max_similitud  # Añadir la similitud más alta
        
        matches.append(best_match_info)  # Añadir la coincidencia a la lista

# Convertir la lista de matches a un DataFrame
df_matches = pd.DataFrame(matches)

# Si deseas mantener todas las filas del proveedor, incluyendo aquellas sin coincidencias, puedes hacer un merge
df_matches_all = pd.merge(df_Proveedor, df_matches, how='left', on='descripcion_limpia_proveedor')

# Reordenar columnas para que 'similaridad' esté al principio
cols = ['similaridad'] + [col for col in df_matches_all.columns if col != 'similaridad']
df_matches_all = df_matches_all[cols]

# Mostrar el resultado
df_matches_all


,similaridad,publicado/\nno publicado_x,u n_x,linea_x,ean_x,cod. dispro_x,cod. elea_x,cod. cid latina_x,registro alfabeta_x,codigo kairos_x,...,ean_y,cod. dispro_y,cod. elea_y,cod. cid latina_y,registro alfabeta_y,codigo kairos_y,pami_y,certif nro_y,descripcion_y,presentacion_y
0,0.154652,PUBLICADO,BIO,Biotecnología,7796285049928,14954,24992,2000601.0,27613,NaN,...,7796285049928,14954,24992,2000601.0,27613,NaN,NaN,48650,3TC + AZT ELEA,Env. x 60 comprimidos
1,0.165141,PUBLICADO,BIO,Biotecnología,7796285050658,17100,25065,2003701.0,32410,NaN,...,7796285050658,17100,25065,2003701.0,32410,NaN,NaN,49022,ABACAVIR ELEA,Env. x 60 comprimidos recub.
2,0.212607,PUBLICADO,BIO,Biotecnología,7796285283551,19341,28355,NaN,56653,NaN,...,7796285283551,19341,28355,NaN,56653,NaN,NaN,58983,ALCAF,60mg F. Amp. Polvo x 1
3,0.290799,PUBLICADO,BIO,Biotecnología,7796285283520,19869,28352,NaN,60305,NaN,...,7796285283520,19869,28352,NaN,60305,NaN,NaN,NaN,BENJOR,50 mg x comp x 30
4,0.333056,PUBLICADO,BIO,Biotecnología,7796285277833,17779,27783,NaN,53125,NaN,...,7796285277833,17779,27783,NaN,53125,NaN,NaN,57934,BEVAX 100 MG,Env x 1 vial de 4 ml.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,0.210294,PUBLICADO,P.C.,Cardio,7796285284442,19431,28444,9008752.0,57299,283302.0,...,7796285284442,19431,28444,9008752.0,57299,283302.0,658271-2,NaN,ZATIUM MET,50/500 MG COMP X 60
698,0.200132,PUBLICADO,P.C.,Cardio,7796285284459,19434,28445,9008750.0,57298,283303.0,...,7796285284459,19434,28445,9008750.0,57298,283303.0,658284-1,NaN,ZATIUM MET,50/850 MG COMP X 30
699,0.213503,PUBLICADO,P.C.,Cardio,7796285284473,19430,28447,9008753.0,57297,283304.0,...,7796285284473,19430,28447,9008753.0,57297,283304.0,658284-2,NaN,ZATIUM MET,50/850 MG COMP X 60
700,0.450756,PUBLICADO,P.C.,Visual,7796285255169,13702,25516,3628302.0,36159,165772.0,...,7796285255169,13702,25516,3628302.0,36159,165772.0,53058410,51699,ZOPIROL,0.50% SOL.OFT. x 5 ML


In [281]:
df_matches_all[["similaridad", 'niprod', 'descripcion_limpia_proveedor', 'descripcion_limpia_producto', 'atrib0']]

,similaridad,niprod,descripcion_limpia_proveedor,descripcion_limpia_producto,atrib0
0,0.154652,91047.0,3 tc azt elea envase x 60 comprimidos,fingras 60 miligramos capsulas x 60,PHOENIX
1,0.165141,91047.0,abacavir elea envase x 60 comprimidos recub,fingras 60 miligramos capsulas x 60,PHOENIX
2,0.212607,91018.0,alcaf 60 miligramos f ampolla polvo x 1,doxtran b 12 inyeccion frasco ampolla x 3,PHOENIX
3,0.290799,26498.0,benjor 50 miligramos x comprimidos x 30,causalon comprimidos x 50,PHOENIX
4,0.333056,91318.0,beva x 100 miligramos envase x 1 vial de 4 mil...,xeomin 100 ui vial x 1,PHOENIX
...,...,...,...,...,...
697,0.210294,91047.0,zatium met 50500 miligramos comprimidos x 60,fingras 60 miligramos capsulas x 60,PHOENIX
698,0.200132,91224.0,zatium met 50850 miligramos comprimidos x 30,pioglit 30 miligramos comprimidos x 30,PHOENIX
699,0.213503,91047.0,zatium met 50850 miligramos comprimidos x 60,fingras 60 miligramos capsulas x 60,PHOENIX
700,0.450756,90987.0,zopirol 050% solucion oftalmica x 5 mililitros,clare x compuesto solucion oftalmica x 5 milil...,PHOENIX


In [282]:
df_matches_all[df_matches_all['similaridad'] >= 0.75][['similaridad', 'niprod', 'descripcion_limpia_proveedor', 'descripcion_limpia_producto', 'atrib0']]

,similaridad,niprod,descripcion_limpia_proveedor,descripcion_limpia_producto,atrib0
470,0.938959,91093.0,loreme x descongestivo comprimidos x 5,loreme x descongestivo comprimidos x 10,PHOENIX
492,0.796588,91147.0,nefazan 75 miligramos x 30 comprimidos rec,nefazan 75 miligramos comprimidos x 60,PHOENIX
513,0.890747,91160.0,neumote x nebu x 20 mililitros,neumote x nebu envase x 20 mililitros,PHOENIX
604,0.787082,91266.0,rabloc 10 miligramos x 30 comprimidos rec,rabloc 5 miligramos comprimidos x 30,PHOENIX
605,0.789408,91266.0,rabloc 20 miligramos x 30 comprimidos rec,rabloc 5 miligramos comprimidos x 30,PHOENIX


In [283]:
# Buscar el valor 102876 en la columna 'niprod'
#resultado = df_MasterProductos_Filtro[df_MasterProductos_Filtro['niprod'] == 16787]

resultado = df_MasterProductos[df_MasterProductos['niprod'] == 91093]
resultado
# Ver el resultado
#resultado['descripcion_limpia_producto'].tolist()



,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descripcion_limpia_producto
32662,91093.0,20310,20310,LOREMEX DESCONGESTIVO comp.x 10,NaN,LOREMEX DESCONGESTIV,PT,7791848079015,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,loreme x descongestivo comprimidos x 10


In [284]:
# Buscar 'allopurinol fabra' en la columna 'descripcion_limpia_producto' (ajusta el nombre de la columna si es diferente)
dddd = df_MasterProductos[df_MasterProductos['descripcion_limpia_producto'].str.contains('DABIDANE', case=False, na=False)]

# Mostrar el resultado
dddd[['descrip1', 'atrib0']]


,descrip1,atrib0
15399,DABIDANE 110 mg cáps.x 30,ELEA
15400,DABIDANE 110 mg cáps.x 60,ELEA
15401,DABIDANE 150 mg cáps.x 30,ELEA
15402,DABIDANE 150 mg cáps.x 60,ELEA
15403,DABIDANE 75 mg cáps.x 30,ELEA


Prueba de  NUEVO ENFOQUE: HIBRIDO


In [291]:

def procesar_descripcion_HIBRIDO(data, nombre_columna):
    """
    Procesa una columna de un DataFrame aplicando normalización y manejo de combinaciones de texto.
    
    Parámetros:
    - data: DataFrame que contiene la columna a procesar.
    - nombre_columna: Nombre de la columna que se va a procesar.
    
    Retorna:
    - DataFrame con la columna procesada y actualizada.
    """
    # Verificar si la columna existe en el DataFrame
    if nombre_columna not in data.columns:
        raise KeyError(f"La columna '{nombre_columna}' no existe en el DataFrame.")
    
    # Función para normalizar el texto
    def normalizar(texto):
        # Asegurar que el texto sea un string
        texto = str(texto).strip().lower()
        
        # 1. Eliminar espacios redundantes
        texto = re.sub(r'\s+', ' ', texto)

        # 2. Remover caracteres especiales excepto letras, números, espacios y '%'
        texto = re.sub(r'[^a-zA-Z0-9\s%]', '', texto)

        # 3. Separar números de letras y letras de números
        texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
        texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)

        # 4. Manejo de 'x':
        texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)  # separar x entre números
        texto = re.sub(r'\b([a-zA-Z]+)x\b', r'\1 x', texto)  # separar palabras terminadas en x

        # 5. Reemplazar separadores como guiones o barras con espacios
        texto = re.sub(r'[-/]', ' ', texto)

        # 6. Manejo de unidades (por ejemplo, separar g, ml, etc. de los números)
        texto = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', texto)

        # 7. Eliminar espacios redundantes nuevamente
        texto = texto.strip()

        # 8. Combinaciones
        combinaciones = {
            'anemido x': 'anemidox',
            'jgaprell' : 'jga prell',
            'fle x': 'flex',
            'argeflo x':'argeflox',
            'aropa x': 'aropax',
            'arolte x' : 'aroltex',
            'atopi x':  'atopix',
            'atorma x': 'atormax',
            'comprec': 'comp recubierto',
            'soloft': 'solucion oftalmica',
            'g': 'gr',
            'cr': 'crema',
            'comp' : 'comprimidos',
            'caps': 'capsulas',
            "jer.pre" : "jeringa prell",
            "mg" : "miligramos",
            "ml" : "mililitros",
            "mm" : "milimetros",
            'zyvo x': 'zyvox',
            'loc' : 'locion',
            "gr" : "gramos",
            "jga" : 'jeringa',
            "cm" : 'centimetros',
            'u' : "unidad",
            'shamp' : 'shampoo',
            "unid" : " unidad",
            'cep' : 'cepillo dental',
            "emuls prot" : 'emulsion protectora',
            'emulshidrat':'emulsion hidratante',
            'emulshumect' : "emulsion humectante",
            'iny': 'inyeccion',
            'amp' : 'ampolla',
            'inylioffa': 'inyeccion lioffa',
            'ds' : 'dosis',
            'inyfa': 'inyeccion fa',
            'jbe': 'jarabe',
            'blist': 'blister',
            'sob' : 'sobres',
            'fa': 'frasco ampolla',
            'jab' : 'jabon',
            'cps' :'capsulas',
            'ivi x' : 'ivix',
            'inya': 'inyectable',
            'inyjgaprell' : 'iny jeringa prellenada',
            'mcg': 'microgramo',
            'gts': 'gotas',
            'mgml' : 'miligramos por mililitro',
            'env': 'envase',
            'gtsoft' : 'gotas oftálmicas',
            'compdisp': 'comprimidos disp',
            'comprecran' : 'comprimidos recubiertos ranurados',
            'sol': 'solucion',
            'jgapre': 'jeringa prellenada',
            'fcocps' : 'frasco capsulas',
            'fcogotero' : 'frasco gotero',
            'fco': 'frasco', 
            'solspray' : 'solucion spray',
            'solpuas' : 'sol puas',
            'cpsblandas' : 'cps blandas',
            'tab' : 'tableta',
            'sach' : 'sachet',
            'mgvial': 'miligramos vial',
            'cpsbl' :'capsula blanda',
            "recub" :"recubiertos",
        }
        for key, val in combinaciones.items():
            texto = re.sub(rf'\b{key}\b', val, texto)

        # Convertir números a float (o int) para que se traten como valores numéricos
        def convertir_a_numero(palabra):
            try:
                return float(palabra) if '.' in palabra else int(palabra)
            except ValueError:
                return palabra
        
        # Aplicar la conversión a cada palabra
        texto = " ".join([str(convertir_a_numero(palabra)) for palabra in texto.split()])

        return texto

    # Aplicar la función 'normalizar' a cada elemento de la columna
    data[nombre_columna] = data[nombre_columna].apply(normalizar)
    
    # Aquí aplicamos el split
    data[nombre_columna] = data[nombre_columna].str.split(' ')

    return data  # Retorna el DataFrame con la columna procesada y la nueva columna de split


In [329]:
#Procesamiento de Columna Descripcion
# Procesamiento de df_MasterProductos: descripcion_limpia_producto

df_MasterProductos_Filtro_hibrido = procesar_descripcion_HIBRIDO(df_MasterProductos_Filtro, "descripcion_limpia_producto")

# Procesamiento de df_Proveedor: descripcion_limpia_proveedor
df_Proveedor_hibrido = procesar_descripcion_HIBRIDO(df_Proveedor, 'descripcion_limpia_proveedor')



C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_14988\47486557.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[nombre_columna] = data[nombre_columna].apply(normalizar)
C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_14988\47486557.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[nombre_columna] = data[nombre_columna].str.split(' ')


In [330]:
df_MasterProductos_Filtro_hibrido

,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descripcion_limpia_producto,numeros
604,26458.0,33228,33228,ACOTRAL 10 mg comp.x 10,NaN,ACOTRAL 10 mg comp.x,PT,7791848286017,NaN,UN,...,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[acotral, 10, miligramos, comprimidos, x, 10]","[1, 0, 1, 0]"
605,26459.0,33229,33229,ACOTRAL 10 mg comp.x 20,NaN,ACOTRAL 10 mg comp.x,PT,7791848286024,NaN,UN,...,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[acotral, 10, miligramos, comprimidos, x, 20]","[1, 0, 2, 0]"
606,26460.0,33751,33751,ACOTRAL 10 mg comp.x 30,NaN,ACOTRAL 10 mg comp.x,PT,7791848286031,NaN,UN,...,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[acotral, 10, miligramos, comprimidos, x, 30]","[1, 0, 3, 0]"
1266,26461.0,33082,33082,AGIOFIBRAS gran.x 250 g,NaN,AGIOFIBRAS gran.x 25,PT,7791848259066,NaN,UN,...,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[agiofibras, gran, x, 250, gramos]","[2, 5, 0]"
1267,26462.0,46479,46479,AGIOFIBRAS sob.x 12,NaN,AGIOFIBRAS sob.x 12,PT,7791848259097,NaN,UN,...,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[agiofibras, sobres, x, 12]","[1, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,91314.0,23342,23342,XANOL 12 mcg caps.x 30,NaN,XANOL 12 mcg caps.x,PT,7791848262028.0,NaN,UN,...,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[xanol, 12, microgramo, capsulas, x, 30]","[1, 2, 3, 0]"
56038,91315.0,25874,25874,XANOL 12 mcg caps.x 60,NaN,XANOL 12 mcg caps.x,PT,7791848262042.0,NaN,UN,...,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[xanol, 12, microgramo, capsulas, x, 60]","[1, 2, 6, 0]"
56039,91316.0,23341,23341,XANOL 6 mcg caps.x 30,NaN,XANOL 6 mcg caps.x 3,PT,7791848262011.0,NaN,UN,...,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[xanol, 6, microgramo, capsulas, x, 30]","[6, 3, 0]"
56040,91317.0,25873,25873,XANOL 6 mcg caps.x 60,NaN,XANOL 6 mcg caps.x 6,PT,7791848262035.0,NaN,UN,...,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[xanol, 6, microgramo, capsulas, x, 60]","[6, 6, 0]"


In [334]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Función para calcular la diferencia numérica
def calcular_diferencia_numerica(prod1, prod2):
    # Extraer solo los números de las descripciones
    prod1_numeros = [float(x[0]) for x in prod1 if x[0].isdigit()]
    prod2_numeros = [float(x[0]) for x in prod2 if x[0].isdigit()]

    # Si ambos productos tienen números, calcular la diferencia
    if prod1_numeros and prod2_numeros:
        # Asegurar que las listas de números tienen la misma longitud
        if len(prod1_numeros) > len(prod2_numeros):
            prod2_numeros.extend([0] * (len(prod1_numeros) - len(prod2_numeros)))
        elif len(prod1_numeros) < len(prod2_numeros):
            prod1_numeros.extend([0] * (len(prod2_numeros) - len(prod1_numeros)))

        # Calcular la diferencia relativa
        diferencia = abs(np.array(prod1_numeros) - np.array(prod2_numeros)) / np.array(prod1_numeros)
        return np.mean(diferencia)
    return 0

# Vectorización de descripciones
descripciones_completas = df_MasterProductos_Filtro_hibrido['descripcion_limpia_producto'].tolist() + df_Proveedor_hibrido['descripcion_limpia_proveedor'].tolist()
vectorizer = TfidfVectorizer().fit_transform(descripciones_completas)

# Calcular similitud de coseno
cos_sim_matrix = cosine_similarity(vectorizer[:len(df_MasterProductos_Filtro_hibrido)], vectorizer[len(df_MasterProductos_Filtro_hibrido):])

matches = []

# Comparar productos de proveedores con productos de la master
for i, prov_row in df_Proveedor_hibrido.iterrows():
    prov_info = prov_row.to_dict()  # Información completa del proveedor
    
    # Similitudes entre el producto del proveedor y todos los productos de la master
    similitudes_texto = cos_sim_matrix[:, i]
    
    # Mejor coincidencia de texto
    max_similitud_texto = np.max(similitudes_texto)
    best_match_index = np.argmax(similitudes_texto)
    
    # Información de la mejor coincidencia de la master
    best_match_info = df_MasterProductos_Filtro_hibrido.iloc[best_match_index].to_dict()
    
    # Comparar las descripciones numéricas
    diferencia_numerica = calcular_diferencia_numerica(prov_info['descripcion_limpia_proveedor'], df_MasterProductos_Filtro_hibrido.iloc[best_match_index]['descripcion_limpia_producto'])
    
    # Calcular similitud final (principalmente numérica)
    similitud_final = 1 - diferencia_numerica  # Solo se basa en la comparación numérica
    similitud_final = similitud_final * 0.9 + max_similitud_texto * 0.1  # Ajuste por similitud de texto
    
    # Unir la información
    best_match_info.update(prov_info)
    best_match_info['similaridad'] = similitud_final
    
    matches.append(best_match_info)

# Convertir matches a DataFrame
df_matches = pd.DataFrame(matches)

# Merge con el DataFrame del proveedor
df_matches_all = pd.merge(df_Proveedor_hibrido, df_matches, how='left', on='descripcion_limpia_proveedor')

# Reordenar columnas para que 'similaridad' esté al principio
cols = ['similaridad'] + [col for col in df_matches_all.columns if col != 'similaridad']
df_matches_all = df_matches_all[cols]

# Mostrar resultados
df_matches_all


AttributeError: 'list' object has no attribute 'lower'

In [312]:
df_matches_all_hibrido

,similaridad,descripcion_master,descripcion_proveedor
1408,0.034779,repetido supportan 500 mililitros caja x 15 sa...,beva x 100 miligramos envase x 1 vial de 4 mil...
1409,0.031838,repetido supportan 500 mililitros caja x 15 sa...,beva x 400 miligramos envase x 1 vial de 16 mi...
1410,0.028684,repetido supportan 500 mililitros caja x 15 sa...,biovac hb 10 microgramo inyectable frasco ampo...
1411,0.028659,repetido supportan 500 mililitros caja x 15 sa...,biovac hb 20 microgramo inyectable frasco ampo...
1424,0.030202,repetido supportan 500 mililitros caja x 15 sa...,dafe x liof 440 miligramos 50 mililitros
...,...,...,...
39852533,0.023172,zz top comprimidos x 30,zatium met 501000 miligramos comprimidos x 60
39852534,0.060202,zz top comprimidos x 30,zatium met 50500 miligramos comprimidos x 30
39852535,0.022992,zz top comprimidos x 30,zatium met 50500 miligramos comprimidos x 60
39852536,0.060333,zz top comprimidos x 30,zatium met 50850 miligramos comprimidos x 30
